# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235234259670                   -0.50    7.0         
  2   -7.250060722333       -1.83       -1.38    1.0   8.20ms
  3   -7.251232767941       -2.93       -2.03    2.0   9.52ms
  4   -7.251265025799       -4.49       -2.15    2.0   9.20ms
  5   -7.251330850279       -4.18       -2.75    1.0   7.98ms
  6   -7.251336456951       -5.25       -2.95    3.0   10.2ms
  7   -7.251338672724       -5.65       -3.64    1.0   8.04ms
  8   -7.251338762610       -7.05       -3.80    3.0   11.6ms
  9   -7.251338796553       -7.47       -4.40    1.0   8.49ms
 10   -7.251338798492       -8.71       -4.99    2.0   72.4ms
 11   -7.251338798672       -9.74       -5.25    3.0   12.0ms
 12   -7.251338798700      -10.56       -5.74    1.0   8.45ms
 13   -7.251338798703      -11.49       -5.96    2.0   9.89ms
 14   -7.251338798704      -11.87       -6.40    1.0   8.34ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06002932645525927
[ Info: Arnoldi iteration step 2: normres = 0.432185353555982
[ Info: Arnoldi iteration step 3: normres = 1.005018773743471
[ Info: Arnoldi iteration step 4: normres = 0.28798328678413115
[ Info: Arnoldi iteration step 5: normres = 0.5829707048385495
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.44e-02, 7.28e-02, 5.18e-01, 2.51e-01, 1.03e-02)
[ Info: Arnoldi iteration step 6: normres = 0.23376690197089559
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.00e-02, 8.04e-02, 1.73e-01, 1.02e-01, 6.51e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08772577385169811
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.27e-04, 6.99e-03, 1.36e-02, 5.48e-02, 6.13e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11153187433705043
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.08e-0